In [8]:
import warnings
warnings.filterwarnings("ignore")
import pathlib
import os
from src.midi_processing import get_song_vector, DOC2VEC_MODELS_PATHS
import numpy as np
from sklearn.manifold import TSNE
import joblib
from tqdm import tqdm
import multiprocessing
import matplotlib.pyplot as plt

models = {name: joblib.load(os.path.join(DOC2VEC_MODELS_PATHS, f'{name}_model.jblib')) for name in ['drums', 'melody', 'harmony']}

In [9]:
midi_files = pathlib.Path('Data/midi_files/')
all_midi_paths = list(midi_files.glob('*'))

In [10]:
songs_vectors = []
song_names = []

for midi_file in tqdm(all_midi_paths, total=len(all_midi_paths)):
    try:
        songs_vectors.append(get_song_vector(str(midi_file), models))
        song_names.append(midi_file.name)
    except Exception as e:
        print(e)
        print(f"Invalid song: {midi_file.name}")
        continue        

  3%|█▍                                                    | 16/624 [00:04<02:33,  3.95it/s]

data byte must be in range 0..127
Invalid song: Aaron_Neville_-_Tell_It_Like_It_Is.mid


 20%|██████████▍                                          | 123/624 [00:43<02:08,  3.91it/s]

Could not decode key with 1 flats and mode 255
Invalid song: Beastie_Boys_-_Girls.mid


 27%|██████████████                                       | 166/624 [00:58<02:05,  3.64it/s]

...

 58%|██████████████████████████████▋                      | 361/624 [02:03<01:15,  3.49it/s]

Could not decode key with 4 flats and mode 255
Invalid song: Dan_Fogelberg_-_Leader_of_the_Band.mid


 59%|███████████████████████████████▏                     | 367/624 [02:05<01:20,  3.19it/s]

Could not decode key with 16 sharps and mode 1
Invalid song: David_Bowie_-_Lazarus.mid


100%|█████████████████████████████████████████████████████| 624/624 [03:35<00:00,  3.69it/s]


In [11]:
songs_vectors = np.array(songs_vectors)

tsne = TSNE(n_components=2, verbose=3, random_state=84)
vectors_transformed = tsne.fit_transform(songs_vectors)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 617 samples in 0.003s...
[t-SNE] Computed neighbors for 617 samples in 0.097s...
[t-SNE] Computed conditional probabilities for sample 617 / 617
[t-SNE] Mean sigma: 5.001674
[t-SNE] Computed conditional probabilities in 0.029s
[t-SNE] Iteration 50: error = 109.3854065, gradient norm = 0.2940904 (50 iterations in 0.201s)
[t-SNE] Iteration 100: error = 115.3806458, gradient norm = 0.2489248 (50 iterations in 0.201s)
[t-SNE] Iteration 150: error = 115.6661301, gradient norm = 0.2597208 (50 iterations in 0.199s)
[t-SNE] Iteration 200: error = 116.8240967, gradient norm = 0.2599342 (50 iterations in 0.214s)
[t-SNE] Iteration 250: error = 118.1528931, gradient norm = 0.2848582 (50 iterations in 0.194s)
[t-SNE] KL divergence after 250 iterations with early exaggeration: 118.152893
[t-SNE] Iteration 300: error = 2.8647766, gradient norm = 0.0051359 (50 iterations in 0.175s)
[t-SNE] Iteration 350: error = 2.2743156, gradient norm = 0.001

In [12]:
from bokeh.plotting import figure, output_file, save
from bokeh.models import ColumnDataSource, HoverTool

# output to static HTML file
output_file("Interactive-Midi-Doc2Vec.html")

data_source = ColumnDataSource(data=dict(x=vectors_transformed[:, 0], y=vectors_transformed[:, 1], song_name=song_names))
hover = HoverTool(tooltips=[
    ("song_name", "@song_name"),
])

p = figure(sizing_mode='stretch_both', title="Melody embedding TSNE visualization")
p.add_tools(hover)


# add a circle renderer with a size, color, and alpha
p.circle(source=data_source, x='x', y='y', size=10, color="navy", alpha=0.5)

# show the results
save(p)

'C:\\Users\\Eli\\Workspace\\Deep-Learning-Intro\\assignment3\\Interactive-Midi-Doc2Vec.html'